In [ ]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
from scipy.signal import find_peaks
from scipy.stats import entropy as ent

from planetary_motion import OrbitalMotion2 as om
from planetary_motion import Coordinates
from planetary_motion import Trig

### Orbital Motion testing

The following code segments demonstrate how to calculate information using the orbital motion equations. 

In [ ]:
# Example from Tutorial
# https://www.stjarnhimlen.se/comp/tutorial.html#5
d = om.date(19, 4.0, 1990)
sun_arr = om.sunPos(d)
merc_arr = om.getPlanPos(1, d)
# Heliocentric
print(merc_arr)
merc_geo = om.togeo(sun_arr, merc_arr)
#print(merc_geo)
mer_geo_eq = Coordinates.ecEq(merc_geo[0], merc_geo[1], merc_geo[2], sun_arr[3])
print(mer_geo_eq[0], mer_geo_eq[1], mer_geo_eq[2])
sphere = Coordinates.sphere(mer_geo_eq[0], mer_geo_eq[1], mer_geo_eq[2])
print(sphere)

In [ ]:
# Using OrbitalMotion2
# https://www.stjarnhimlen.se/comp/tutorial.html#5
d = om.date(19, 4.0, 1990)
sun_arr = om.getSunData(d = 19, m = 4, y = 1990, ct = "rect", csys = "eclip")
print(sun_arr)
merc_arr = om.getPlanData(d = 19, m = 4, y = 1990, p = "mercury", ct="sphere", geoc=True)
print(merc_arr)

In [ ]:
d = om.date(19, 4.0, 1990)
sun_arr = om.sunPos(d, csys="pertur")
moon_arr = om.getPlanPos(0, d, ct="sphere")
# Geocentric Longitude and Latitude
print("Long/Lat", Trig.rev(moon_arr[0]), moon_arr[1], moon_arr[2])

# RA and Decl (Manual Calculation)
rect = Coordinates.rect(moon_arr[0], moon_arr[1])
equat = Coordinates.ecEq(rect[0], rect[1], rect[2], sun_arr[3])
sphere = Coordinates.sphere(equat[0], equat[1], equat[2])
print('RA = ', Trig.rev(sphere[0]), "Decl = ", sphere[1])

# ALTERNATIVELY:
sphere = om.getPlanData(d=19, m=4.0, y=1990, p = "moon", ct="sphere", csys="equat")
print('RA = ', Trig.rev(sphere[0]), "Decl = ", sphere[1])

LST = om.sidereal(0, sun_arr[0], 15)
azi = Coordinates.azimuthal(sphere[0], sphere[1], LST, 60)
print("Azi", azi)
print(azi[1] - Trig.asind(1/moon_arr[2])*Trig.cosd(azi[1]))

In [ ]:
sun_arr = om.sunPos(d, csys="equat", ct="sphere")
print(sun_arr[0], sun_arr[1])
azi = Coordinates.azimuthal(sun_arr[0], sun_arr[1], LST, 60)
print(azi)

In [ ]:
"""
    Collecting data for two planets starting at 
    date value d and continuing for const days
"""
d = 0.0
const = 365*13
# x,y,z equatorial coordinates
p1 = np.zeros((3, const))
p2 = np.zeros((3, const))
for i in range (const):
    sun_arr = om.sunPos(d)
    p1_arr = om.getPlanPos(1, d)
    p2_arr = om.getPlanPos(4, d)
    
    p1_geo = om.togeo(sun_arr, p1_arr)
    p1_geo_eq = Coordinates.ecEq(p1_geo[0], p1_geo[1], p1_geo[2], sun_arr[3])
    p1[0][i] = p1_geo_eq[0]
    p1[1][i] = p1_geo_eq[1]
    p1[2][i] = p1_geo_eq[2]

    p2_geo = om.togeo(sun_arr, p2_arr)
    p2_geo_eq = Coordinates.ecEq(p2_geo[0], p2_geo[1], p2_geo[2], sun_arr[3])
    p2[0][i] = p2_geo_eq[0]
    p2[1][i] = p2_geo_eq[1]
    p2[2][i] = p2_geo_eq[2]
    d += 1

### WXML testing: Planetary Signs Correlation
FFT of the distance between planets in geocentric orbit. It may be interesting to calculate
an explicit correlation value in the future.

In [ ]:
plt.plot(p1[0], p1[1])
plt.plot(p2[0], p2[1])
plt.title("Geocentric orbit of Jupiter and Mercury in 2D for 13 years")
plt.ylabel("Location from Earth in astronomical units (y)")
plt.xlabel("Location from Earth in astronomical units (x)")

In [ ]:
mag = np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 + (p1[2] - p2[2]) ** 2 )
plt.plot(np.arange(0, const), mag)
plt.title("Magnitude of distance between Jupiter and Mercury (au) vs Time (days)")
plt.ylabel("Planet distance in astronomical units")
plt.xlabel("Days elapsed since Dec 31st, 1999")


print(find_peaks(mag))

In [ ]:
y = fft(mag)
z =np.log(abs(y))
plt.plot(z)
z1 = find_peaks(z)
print(z1)
plt.title("Log of Fourier Transform of the distance between Jupiter and Mercury")
plt.ylabel("Intensity")
plt.xlabel("Frequency")

In [ ]:
# RA temp-func
def temp_func(n, const):    
    d = 0.0
    # x,y,z equatorial coordinates
    p1_0 = np.zeros((3, const))
    #sun_0 = np.zeros((2, const))
    for i in range (const):
        temp = om.getPlanPosGeo(n, d, ct="sphere", csys="equat")
        p1_0[0][i] = temp[0]
        p1_0[1][i] = temp[1]
        d += 1

    return p1_0

### WXML Testing - RA Distribution
Code for displaying the right ascension angle distribution of planets and the frequency of their appearance.

In [ ]:
# Distribution of RA angles for Mercury (we hope)
const = 365
p1_0 = temp_func(1, const)

plt.hist(p1_0[0], bins = [-180, -150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150, 180])
plt.title("Distribution of planetary signs for Mercury for 1 year")
plt.xlabel("Range of Right-Ascension angle (in degrees) representing 1 planetary sign")
plt.ylabel("Days Mercury lies within this range")
plt.show()

#print(p1_0[0])

In [ ]:
# Distribution of RA angles to Venus on 1 day for 50 years
const = 50*365
p1_0 = temp_func(2, const)
p1_2 = np.zeros(50)

# Get the nth year's value
for i in range(50):
    v = 365 * i
    p1_2[i] = p1_0[0][v]

plt.hist(p1_2, bins = [-140, -130, -115, -100, -90, -80, -70, -60, -50, -40, -30])
plt.title("Distribution of Right-Ascension angles of Venus on 1 day for 50 years")
plt.xlabel("Range of Right-Ascension angle (in degrees)")
plt.ylabel("Days Venus lies within this range")
plt.show()

In [ ]:
# Numerical data on distribution values

p1_hist = np.histogram(p1_2, bins=[-140, -130, -115, -100, -90, -80, -70, -60, -50, -40, -30])

for i in range(len(p1_hist[0])):
    print("Days Mercury lies in the sign of range [" +  str(p1_hist[1][i]) + "," + str(p1_hist[1][i+1]) + ") degrees: ", p1_hist[0][i])
p1_hist

# Make sure all data is covered within the provided range
sum = 0
for i in range(len(p1_hist[0])):
    sum += p1_hist[0][i]

print(sum)

### WXML Testing -- Entropy
Code for calculating entropy for these RA values and the displaying results.

In [ ]:
'''
    A quick method for calculating the entropy of a system with discrete probabilities.
    We will check our calculated values with the scipy implementation.
    @param pdist: probability distribution of the events in a system
'''

def ent_calc(pdist):
    entropy = 0
    print("Rounded probabilities:", end=' ')
    for i in range(len(pdist)):
        px = pdist[i]
        # Probabilities (rounded)
        print("%.2f" %(px), end=' ')
        if px != 0:
            entropy += -px * np.log2(px)
    return entropy

In [ ]:
# Our calculation
entval = ent_calc(p1_hist[0]/50)
print('\nCalculated Entropy:', entval)

# Scipy method 
print('Calculated Entropy (scipy method):', ent(p1_hist[0]/50, base=2))

# Normalized Value
print('Normalized Value:', entval/np.log2(len(p1_hist[0])))
# Max Value of Entropy
print('Max Entropy:', np.log2(len(p1_hist[0])))

### WXML Testing -- Generated Keys
Code used to print decimal values to a file and for calculating entropy of final codes.

In [ ]:
# Take the first 4 decimal values of the RA angle over whatever you're iterating over
# This sample is taking for at approximately the same day at 25 year intervals
# Sample codes: 2325 0249 8162 6063
scripts = []
for i in range(0, 93, 1):
    temp = str (object=p1_0[0][i])
    scripts.append(temp[(temp.index('.'))+1:(temp.index('.'))+5])
    #print(temp[(temp.index('.'))+1:(temp.index('.'))+5])
    
    for j in range(1,4):
        temp1 = str (object=p1_0[0][i+(365*25*j)])
        scripts[len(scripts)-1] = scripts[len(scripts)-1] + temp1[(temp1.index('.'))+1:(temp1.index('.'))+5]

In [ ]:
# Export the codes to a file
f = open('25yrjump.txt', 'w')
for i in range(len(scripts)):
    f.write(scripts[i])
    f.write('\n')
f.close()

In [ ]:
# Excerpt of randomized strings via Von-Neumann
# Increment each day and collect over 4 days
st1 = "01001010111011010000011000010000000001101100101001010101110000000010000010101010010010110000001011100100000011000100000001001000"
# Increment by each month and collect over 4 months
st2 = "01001000100010010000100001001100101010101000100001000101000100001110110110000001010101101010100101000100000000011010000110010001"
# Increment by 25 years and collect over 100 years
st3 = "01001110000101010110000100010011101100101010000010000001000101000110110001000101011011011000110101011001000100110000011000101110"

gen_keys = np.array([st1, st2, st3])

In [ ]:
'''
    Returns percentage distribution of 1's and 0's appearing in 
    Randomized string
'''
def sumBinarys(text):
    zeros = 0
    ones = 0
    for i in range(len(text)):
        if text[i] == '0':
            zeros+=1
        else:
            ones+=1

    return np.array([zeros/len(text), ones/len(text)])

In [ ]:
# Entropy values for generated keys
for i in range(3):
    vals = sumBinarys(gen_keys[i])
    print("Percentage of zeros: ", vals[0])
    print("Percentage of ones: ", vals[1])
    print("Calculated entropy: ", ent(vals, base=2))